In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


In [57]:
# Set a random seed for reproducibility
np.random.seed(0)

In [58]:
# Read the CSV file
df = pd.read_csv('Default.csv')
df.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879


In [59]:
# Assuming df is your DataFrame and 'default' is the target column
X_a = df[['income', 'balance']]
y_a = df['default']



In [60]:
# i. Split the data into training and validation sets
X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_a, y_a, test_size=0.2, shuffle=True, random_state=17)

In [61]:
# ii. Fit a SVM on the training set
clf = svm.SVC(kernel='linear')
clf.fit(X_train_a, y_train_a)

SVC(kernel='linear')

In [62]:
# iii. Obtain a prediction of default status for validation set
y_val_pred_a = clf.predict(X_val_a)

In [63]:
# iv. Compute the validation set error.
val_error_a = 1 - accuracy_score(y_val_a, y_val_pred_a)
print(f"Validation set error for Part (a): {val_error_a}")

Validation set error for Part (a): 0.03049999999999997


In [64]:
# (b) 5-fold cross validation
kf = KFold(n_splits=5, random_state=29, shuffle=True)

In [65]:
# Function to calculate validation error
def calculate_error_a(X, y, clf):
    errors = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        val_error = 1 - accuracy_score(y_val, y_val_pred)
        errors.append(val_error)
    return np.mean(errors)

In [66]:
# Calculate validation errors for (b)
val_error_a_cv = calculate_error(X_a, y_a, clf)


print("[a] 5-fold CV validation error : ", val_error_a_cv)


[a] 5-fold CV validation error :  0.031200000000000006


(a)에서는 데이터에 단일 검증 세트를 분할해서 모델 성능을 추정했다. 다만 (a)는 과적합 또는 과소적합으로 인한 문제가 발생할 수 있으므로 새로운 데이터에 대한 모델 성능의 신뢰도가 낮다.
(b)에서는 동일한 데이터 세트에서 모델을 5개의 하위 fold로 나누고 모델을 5번 훈련하고 평가한다. 5 fold cross validation은 여러 validation set에 대한 평균을 계산하여 특정 split의 특성이 error에 미치는 영향을 줄임으로써 모델 성능이 안정적이고 신뢰할만하다.
하지만 (a)의 validation set error와 (b) 5 fold validation error의 차이가 크지 않다. 이는 두 모델 모두 다양한 데이터 세트에서 상대적으로 안정적이고 일관적임을 나타낸다.

In [67]:
# (c) SVM that predicts default using income, balance, and a dummy variable for student
le = LabelEncoder()
df['student_dummy'] = le.fit_transform(df['student'])

X_c = df[['income', 'balance', 'student_dummy']]
y_c = df['default']

In [68]:
# Split the data into training and validation sets
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_c, y_c, test_size=0.2, shuffle=True, random_state=29)

In [70]:
# Calculate validation errors for (c)
val_error_c_cv = calculate_error(X_c, y_c, clf)
print("[c] 5-fold CV validation error : ", val_error_c_cv)

[c] 5-fold CV validation error :  0.031200000000000006


학생의 더미 변수를 포함한 모델과 포함하지 않은 모델을 비교하였을 때
(a) SVM 모델의 validation error 는 0.0305
(c) 더미 변수가 포함된 모델의 validation error는 0.0312
두 모델의 validation error가 매우 비슷하며, 이 경우 학생에 대한 더미 변수를 포함해도 테스트 오차율이 크게 감소하지 않은 것으로 보인다. cross validation 과정에서 볼 수 있듯이 오차 간 미세한 차이는 데이터의 하위 집합을 달리 했을 경우의 오차 변동 범위 내에 일을 수 있다.

더미 변수 추가와 같은 추가 기능을 포함하는 것은 모델의 성능에 미치는 영향은 데이터 집합과 기능과 대상 변수 간의 기본 관계에 따라 달라질 수 있다. 해당 사례에서는 유사한 오차율에서 알 수 있듯이 더미 변수를 포함해도 모델의 예측 성능이 크게 향상되지 않았다.